# Demo.ipynb

Written by Taiob Ali  
SqlWorldWide.com

This script will create:

- A resource group
- One logical Azure SQL server
- Multiple empty Azure SQL Databases
- One Storage Account
- One Storage Container
- Azure Migrate Service
- Azure Database Migration Service

Reference I took help from:

- [Azure PowerShell samples for Azure SQL Database and Azure SQL Managed Instance](https:\docs.microsoft.com\en-us\azure\azure-sql\database\powershell-script-content-guide?tabs=single-database)
- [PowerShell One Liner: Get External/Public IP Address](https:\gallery.technet.microsoft.com\scriptcenter\Get-ExternalPublic-IP-c1b601bb)
- [Azure SQL Database and Azure Synapse IP firewall rules](https:\docs.microsoft.com\en-us\azure\azure-sql\database\firewall-configure#manage-firewall-rules-using-azure-cli)
- [Use PowerShell to create a managed instance](https:\docs.microsoft.com\en-us\azure\azure-sql\managed-instance\scripts\create-configure-managed-instance-powershell)

Pre-Requsite:

- Install Visual Studio
- Install Azure Data Studio
    - Azure SQL Migration Extension installed and enabled
    - SQL Server Dacpac Extension installed and enabled
- Create 'Azure Database Migration Service'  with Self-hosted Integration Runtime
- Azure storage explorer
- sqlpackage installed
- Reuired AZ Module installed
- Restore AdventureWorks database on a local SQL 2022 using backup from [here](.\.%20https:%5Clearn.microsoft.com%5Cen-us%5Csql%5Csamples%5Cadventureworks-install-configure)
- Create a backup of Source database with checksum to the above storage container
- Create a Azure SQL virtual machine
    - Make approprite inbound port rules
- Install Data Migration Assistant (Deprecated)

PowerShell 7.0.6 LTS and PowerShell 7.1.3 or higher is the recommended version of PowerShell for use with the Az PowerShell module on all platforms.  
Check your PowerShell version and update as required

In [ ]:
$PSVersionTable.PSVersion

If you've disabled module autoloading, manually import the module with `Import-Module -Name Az`. Because of the way the module is structured, this can take a few seconds.

In [ ]:
Import-Module -Name Az

Sign in to Azure

If you need to see the list of your subscription

```
$SubscriptionList=Get-AzSubscription$SubscriptionList
```

In [ ]:
Update-AzConfig -EnableLoginByWam $false
$subscriptionId = "6f8db000-8416-43f0-a2db-cbfb7c945982"
$tenantId = "d5b50601-6698-4f8f-beb6-1799fee4dc80"
Connect-AzAccount -DeviceCode -TenantId $tenantId -SubscriptionId $subscriptionId 

Declare Variables

In [ ]:
$resourceGroupName = "taliresourcegroup"
$location = "East US2" 
$sqlServerName = "talisqlserver"
$databaseDma = "databaseDma"
$databaseVS = "databaseVisualStudio"

$azureMigrateName = "taliAzureMigrate"
$vNetName = "migrationServiceVnet"
$databaseMigrationService = "talidms"

$storageAccountName = "talistorageaccount"
$storageContainer = "talibacpac"
$StorageKeyType = "StorageAccessKey"
$databasebacpac_1 = "taliDatabasebacpac1"
$databasebacpac_2 = "taliDatabasebacpac2"

$ipinfo = Invoke-RestMethod http://ipinfo.io/json 
$startip = $ipinfo.ip
$endip = $ipinfo.ip 
$adminlogin = "taiob"
$password = Get-Content "C:\password.txt"

Check if Resource group exists. If exist delete the resource group.

Create a new Resource group

In [ ]:
$resGrpChk = Get-AzResourceGroup `
    -Name $resourceGroupName `
    -ErrorVariable notPresent `
    -ErrorAction SilentlyContinue

if ($resGrpChk) {  
    #Delete resource group
    Remove-AzResourceGroup `
        -Name $resourceGroupName `
        -Force
    Write-Host 'Resource group deleted' `
        -fore white `
        -back green
}

New-AzResourceGroup `
-Name $resourceGroupName `
-Location $location   
Write-Host 'Resource group created' `
    -fore white `
    -back green


Create a Azure SQL Server at Resource Group rgeuazbootcamp (East US)

In [ ]:
New-AzSqlServer `
  -ResourceGroupName $resourceGroupName `
  -ServerName $sqlServerName `
  -Location $location `
  -SqlAdministratorCredentials $(New-Object -TypeName System.Management.Automation.PSCredential `
  -ArgumentList $adminlogin, $(ConvertTo-SecureString -String $password -AsPlainText -Force)) 

Configure Firewall rule for the above server

In [ ]:
New-AzSqlServerFirewallRule `
  -ResourceGroupName $resourceGroupName `
  -ServerName $sqlServerName `
  -FirewallRuleName "taiobDemoMachine" `
  -StartIpAddress $startip `
  -EndIpAddress $endip

# This is done to allow access to Azure Services
New-AzSqlServerFirewallRule `
  -ResourceGroupName $resourceGroupName `
  -ServerName $sqlServerName `
  -FirewallRuleName "AllowAzureService" `
  -StartIpAddress 0.0.0.0 `
  -EndIpAddress 0.0.0.0

Creating an Azure SQL Database to use with Data Migration Assistant

Puchase Model: vCpu  
Service Tier : General Purpose

In [ ]:
New-Azsqldatabase  `
  -ResourceGroupName $resourceGroupName `
  -ServerName $sqlServerName `
  -DatabaseName $databasedma `
  -RequestedServiceObjectiveName "GP_Gen5_2" `
  -MaxSizeBytes 5368709120

Creating an Azure SQL Database to use with Azure Database Migration Service

Puchase Model: vCpu  
Service Tier : General Purpose

In [ ]:
New-Azsqldatabase  `
  -ResourceGroupName $resourceGroupName `
  -ServerName $sqlServerName `
  -DatabaseName $databaseMigrationService `
  -RequestedServiceObjectiveName "GP_Gen5_2" `
  -MaxSizeBytes 5368709120

Creating an Azure SQL Database to use with Visual Studio Project

Puchase Model: vCpu  
Service Tier : General Purpose

In [ ]:
New-Azsqldatabase  `
  -ResourceGroupName $resourceGroupName `
  -ServerName $sqlServerName `
  -DatabaseName $databaseVS `
  -RequestedServiceObjectiveName "GP_Gen5_2" `
  -MaxSizeBytes 5368709120

Create storage account

In [ ]:
$storageaccount = New-AzStorageAccount `
-ResourceGroupName $resourceGroupName `
-AccountName $storageAccountName `
-Location $location `
-SkuName Standard_GRS `
-AllowBlobPublicAccess $true 

Create a Azure Storage Container

In [ ]:
New-AzStorageContainer -Name $storageContainer -Context $storageaccount.Context -Permission Container

Create Azure Migrate Project

Note: For location I am not using the variable. This service is not available in East US

In [ ]:
New-AzMigrateProject  `
-Location "Central US" `
-Name $azureMigrateName  `
-ResourceGroupName $resourceGroupName

Create an Azure Database Migration Service  
This will create a classic one which will be deprecated. I am working on to update this section to create the new <span style="color: var(--vscode-foreground);">Azure Database Migration Service.</span>

[New-AzDataMigrationSqlService](https:\learn.microsoft.com\en-us\powershell\module\az.datamigration\new-azdatamigrationsqlservice?view=azps-11.1.0)

[Migrate databases at scale using automation (Preview)](https:\learn.microsoft.com\en-us\azure\dms\migration-dms-powershell-cli)

In [ ]:
 New-AzDataMigrationSqlService `
 -ResourceGroupName  $resourceGroupName `
 -SqlMigrationServiceName $databaseMigrationService `
 -Location $location

# Demo Starts here

## Show Assessment and Migration:

- SQL Server Management Studio
    
    - Deploy a Database By Using a DAC
- Azure Database Migration Service
    
    - Migrate on-premises database to Azure to Azure SQL Database (offline only)
    - Import Schema from On-premises database
        - Use "SQL Server Dacpac" extension from Azure Data Studio
    - <span style="font-size: 14px;">Export data to above schema only database</span>
    - <span style="color: var(--vscode-foreground);">Improt data from On-premises database</span>
        - <span style="color: var(--vscode-foreground);">Migrate on-premises database to Azure SQL Virtual Machine (offline)</span>
        - <span style="color: var(--vscode-foreground);">Migrate on-premises database&nbsp;to Azure SQL Virtual Machine&nbsp;(online)</span>
- Azure Data Studio Extension "Azure SQL Migration"
    
    - Migrate on-premises database to a Azure SQL Database (offline only)
        - Import Schema from On-premises database
        - Use "SQL Server Dacpac" extension from Azure Data Studio
    - Migrate on-premises database to Azure SQL Virtual Machine (offline)
    - Migrate on-premises database to Azure SQL Virtual Machine (online)
- SqlPackage generate bacpac file
    
    - bacpac file to Azure storage 
    - bacpac file to Local storage
    - Restore from Local storage
    - Restore from Azure storage
- Visual Studio Project (Schema only)
    
    - While publishing under 'Advanced Publish Settings' enable 'Allow incompatible platform'
    - For smaller databases you can do data migration using tools--\>SQL Server--\>New Data Comparision.
- Data Migration Assistant (Deprecated)
    
    - Assessment
        - Export to Azure Migrate
    - Migration

### Use below code while showing online migration
```
USE master;
GO

/*
Set database to full recovery model
*/
ALTER DATABASE AdventureWorks2022 SET RECOVERY FULL;
GO

/*
Will create the table after taking full back and before running transactoin log backup.
*/
USE [AdventureWorks2022];
GO
DROP TABLE IF EXISTS [dbo].[migrationTest];
GO

/*
Take a full backup to blob storage account
*/
BACKUP DATABASE [AdventureWorks2022] 
TO  URL = N'https://talistorageaccount.blob.core.windows.net/talibacpac/adventureworks2022_backup.bak' WITH NOFORMAT, NOINIT,  NAME = N'AdventureWorks2022-Full Database Backup', NOSKIP, NOREWIND, NOUNLOAD,  STATS = 10;
GO

/*
Start a online migration from ADS
*/

/*
Create a table to confirm the transaction logs are restored
*/
USE [AdventureWorks2022];
GO
SET ANSI_NULLS ON;
GO
SET QUOTED_IDENTIFIER ON;
GO
CREATE TABLE [dbo].[migrationTest](
	[test] [smallint] NULL
) ON [PRIMARY];
GO

/*
Take a log backup to blob storage account
*/

BACKUP LOG [AdventureWorks2022]
TO  URL = N'https://talistorageaccount.blob.core.windows.net/talibacpac/adventureworks2022_tlog_backup.trn' 
WITH NOFORMAT, 
NAME = N'AdventureWorks-transaction log Backup', NOSKIP, NOREWIND, NOUNLOAD,  STATS = 10;
GO

/*
Cutover the the migration once last log in restore
Check if dbo.migrationTest table exist
If required set the database to simple recovery model
ALTER DATABASE AdventureWorks2022 SET RECOVERY SIMPLE;
GO
*/
```

Changing to folder where .sqlpackage binary is  
Exporting database to a .bacpac file to local storage

In [ ]:
# Delete file if exists
$FileName = "C:\temp\aw_bacpac.bacpac"

if (Test-Path $FileName) {
  Remove-Item $FileName
  write-host "$FileName has been deleted"
}

else {
  Write-host "$FileName doesn't exist"
}

In [ ]:
SqlPackage.exe /Action:Export /SourceServerName:"taiob2" `
 /SourceTrustServerCertificate:True `
 /SourceDatabaseName:AdventureWorks2022 `
 /TargetFile:"C:\temp\aw_bacpac.bacpac" 

- Copying the bacpac file to storage container using [Azure Storage Explorer](https://azure.microsoft.com/en-us/features/storage-explorer/)
- You can do the same with [AzCopy](https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-v10)
- Restoring Database using bacpac file from storage container

In [ ]:
$StorageUri = "https://talistorageaccount.blob.core.windows.net/talibacpac/aw_bacpac.bacpac"
$StorageKey = (Get-AzStorageAccountKey -ResourceGroupName $ResourceGroupName -AccountName $storageAccountName)[0].Value
$securePassword = ConvertTo-SecureString -String $password -AsPlainText -Force

$importRequest = New-AzSqlDatabaseImport `
  -ResourceGroupName $ResourceGroupName `
  -ServerName $sqlServerName  `
  -DatabaseName $databasebacpac_1 `
  -StorageKeytype $StorageKeyType `
  -StorageKey $StorageKey `
  -StorageUri $StorageUri `
  -AdministratorLogin $adminlogin `
  -AdministratorLoginPassword $securePassword `
  -Edition Standard `
  -ServiceObjectiveName "S4" `
  -DatabaseMaxSizeBytes 5368709120

In [ ]:
Get-AzSqlDatabaseImportExportStatus -OperationStatusLink $importRequest.OperationStatusLink

Restoring Database using bacpac file from local storage

Change the Azure SQL Server Name

In [ ]:
sqlpackage.exe /a:Import /sf:C:\temp\aw_bacpac.bacpac /tsn:talisqlserver.database.windows.net `
/tdn:$databasebacpac_2 `
/tu:taiob@talisqlserver /tp:$password

Cleaning up all resources by deleting the resource group

In [ ]:
Remove-AzResourceGroup -ResourceGroupName $resourceGroupName -Force